# TrackCarbon Estimation Development Notebook

### 🧱 Contributing: Implement Your Own Estimation Methods

This notebook provides the **setup and utilities** needed to **develop, test, and validate**
your own estimation methods.

We currently support **two categories of methods**:

#### ⚡ Energy Estim
ation
Methods that estimate **energy consumption**.

- Must implement `BaseEnergyEstimator`
- Focus on modeling energy usage from inputs (model, tokens, hardware, etc.)

#### 🌍 Energy → Carbon Conversion
Methods that convert **energy estimates into carbon emissions**.

- Must implement `BaseEnergyToCarbonConvertor`
- Focus on carbon intensity, geography, and time dependencies

## 🗺️ Notebook Outline

- **1) Setup & Imports**  
  Imports and events setup. We use the same "fake events" across development. Those are initialised by calling the method `fake_events()`, that returns a list of initialised `Event`. Pls use those when working on your new methods.

- **2) Energy Estimation**  
  - 2.1) Import and test already implemented `NidhalEstimationMethod`
  - 2.2) Implement a dummy new method and test it

- **3) Carbon Conversion**  
  - 3.1) Instanciate a mocked `BaseCarbonIntensityRepository`
  - 3.2) Import and test already implemented `ElectricityMapsBasedConvertor`
  - 3.3) Implement a dummy new method and test it

# 1) Setup & Imports

In [ ]:
from src.domain.models.event import Event
from pathlib import Path
from src.domain.ports.energy_estimator import BaseEnergyEstimator
from src.infrastructure.adapters.estimation.energy.nidhal import NidhalEnergyEstimator
from src.infrastructure.adapters.fake_events import fake_events, fake_events_with_energy
from src.infrastructure.adapters.estimation.carbon.electricity_maps_based_convertor import ElectricityMapsBasedConvertor
from src.infrastructure.adapters.on_disk_carbon_intensity_repository import OnDiskCarbonIntensityRepository
from src.domain.models.event import EventWithEnergy
from src.domain.ports.energy_to_carbon_convertor import BaseEnergyToCarbonConvertor
from src.infrastructure.adapters.estimation.energy.config_loader import ModelConfig

events = fake_events()

# 2) Energy Estimation

## 2.1) Import and test already implemented `NidhalEstimationMethod`

In [ ]:
config = ModelConfig.from_yaml(
    path=Path("src/infrastructure/configs/models/gpt-4o.yaml")
)
event = events[1]  # openai event

nidhal_estimator = NidhalEnergyEstimator(**config.estimator_configs[0].parameters)

In [ ]:
nidhal_estimator.compute(events=[event])

## 2.2) Implement a dummy new method and test it

In [ ]:
class MyEstimator(BaseEnergyEstimator):
    def compute(self, events: list[Event]) -> float:
        return len(events)

    @property
    def method(self) -> str:
        return "my_method"

In [ ]:
estimator = MyEstimator()
estimator.compute(events=fake_events())

# 3) Carbon Estimation

## 3.1) Instanciate a mocked `BaseCarbonIntensityRepository`

We internally have access to some carbon historical carbon intensity data that we summarize by some statistics. We have a dataframe with the following shape including each zone we have access to: FR, 

**Preview (head & tail)**

| month | weekday | zone | mean_carbon_intensity |
|-------|---------|------|------------------------|
| 1     | 0       | FR   | 52.3                   |
| 1     | 1       | FR   | 49.8                   |
| ...   | ...     | ...  | ...                    |
| 12    | 6       | US-CAL-CISO   | 405.6                  |

**DataFrame coverage**

- `month` ranges from **1 → 12**
- `weekday` ranges from **0 (Monday) → 6 (Sunday)**
- `zone` includes **multiple electricity markets** (e.g. `FR`, `US-CAL-CISO` etc...)

**The values have been randomly modified!**

In [ ]:
convertor = ElectricityMapsBasedConvertor(
    carbon_intensity_repository=OnDiskCarbonIntensityRepository(
        file_path="data/carbon_intensity_data_mock.csv"
    )
)

In [ ]:
openai_event = fake_events_with_energy()[1]
anthropic_event = fake_events_with_energy()[0]
openai_value = convertor.to_carbon_emissions(events=[openai_event])
anthropic_value = convertor.to_carbon_emissions(events=[anthropic_event])
print(f"Mistral value: {anthropic_value}")
print(f"Openai value: {openai_value}")

## 3.2) Implement a dummy new method and test it

In [ ]:
class MyConvertor(BaseEnergyToCarbonConvertor):
    def to_carbon_emissions(self, events: list[EventWithEnergy]) -> float:
        """
        Simple convertor that multiplies energy consumed by 2.
        """
        result = 0.0
        for event in events:
            result += event.energy_consumed_kwh * 2
        return result  # gCO2eq

In [ ]:
my_convertor = MyConvertor()
value = my_convertor.to_carbon_emissions(events=[openai_event])
print(value)